## Script to move raw MACS files based on extracted footprints files 
**author:** Ingmar Nitze, Tabea Rettelbach, Simon Schäffler

**contact:** ingmar.nitze@awi.de

**version date:** 2022-02-08

**repository and other tools** https://github.com/awi-response/MACS_tools

### Imports 

In [ ]:
import geopandas as gpd
import shutil
import os
import glob
import pandas as pd
from IPython.display import clear_output
import sys
import numpy as np
import tqdm
import zipfile
from pathlib import Path
from joblib import delayed, Parallel, wrap_non_picklable_objects
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from processing_utils import *

In [110]:
# check for multilayer
def rescale(array, minimum, maximum, dtype=np.uint16, gain=1.):
    factor = 2**16 / maximum
    out = (array - minimum) / (maximum - minimum) * factor * maximum * gain
    out2 = out.round()
    return np.array(np.around(np.clip(out2, 1, (2**16)-1)), dtype=dtype) 

In [117]:
# add tag
def write_new_values(image, minimum, maximum, shutter_factor=1, tag=True):
    with rasterio.open(image, mode='r+')as src:
        data = src.read()
        datanew = rescale(data, minimum, maximum, gain=shutter_factor)
        src.write(datanew)
        if tag:
            src.update_tags(VALUE_STRETCH_MINIMUM=minimum, VALUE_STRETCH_MAXIMUM=maximum)

In [ ]:
def read_stats(image):
    with rasterio.open(image) as src:
        a = src.read()
        return a.mean(), a.min(), a.max(), a.std()

In [ ]:
def read_stats_extended(image):
    with rasterio.open(image) as src:
        a = src.read()
        return a.mean(), a.min(), a.max(), a.std(), np.percentile(a, 1), np.percentile(a, 99)

In [86]:
def get_image_stats_multi(OUTDIR, sensors, n_jobs=40, nth_images=1, quiet=False):
    dfs = []
    for sensor in sensors:
        images = list(OUTDIR[sensor].glob('*.tif'))[::nth_images]
        if quiet:
            stats = Parallel(n_jobs=n_jobs)(delayed(read_stats_extended)(image) for image in images)
        else:
            stats = Parallel(n_jobs=n_jobs)(delayed(read_stats_extended)(image) for image in tqdm.tqdm_notebook(images))
        #stats = [read_stats_extended(image) for image in tqdm.tqdm_notebook(images)]
        df_stats = pd.DataFrame(data=np.array(stats), columns=['mean', 'min', 'max', 'std', 'p01', 'p99'])
        df_stats['image'] = images
        df_stats['sensor'] = sensor
        dfs.append(df_stats)
    df = pd.concat(dfs)
    return df
    

In [130]:
def get_shutter_factor(OUTDIR, sensors):
    """
    get shutter ratio between RGB and NIR. scaling factor for RGB (>1 = increase in values) 
    """
    shutter = {}
    for sensor in sensors:
        images = list(OUTDIR[sensor].glob('*.tif'))
        f = images[0]
        shutter[sensor] = int(f.stem.split('_')[-1])
    if ('right' in sensors) and ('nir' in sensors):
        factor = shutter['nir'] / shutter['right']
    elif ('left' in sensors) and ('nir' in sensors):
        factor = shutter['nir'] / shutter['left']
    else:
        factor = 1
    return factor

In [ ]:
pwd = Path(os.getcwd())

### Functions 

### Settings 
* prefer full/absolute paths
* Create processing template automatically

#### Variable Settings 

In [ ]:
# Set project directory here, where you want to process your data
#PROJECT_DIR = r''
PROJECT_DIR = r'D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_non_cropped' # SET Project output

# Set raw data dir here for the speicific image acquisition project
#path_infiles = r''
path_infiles = r'N:\response\Restricted_Airborne\MACs\Alaska\ThawTrend-Air_2019\raw_data\20190727-235440_15L_Ketik_fire_flight_plan_v3'

# determine which sensors to include in processing (possible options: 'left', 'right', 'nir')
sensors = ['left', 'right', 'nir']

# Set CROP CORNER if 
CROP_CORNER = 0 # SET to 1 if you want to crop corners (set to NoData)
DISK_SIZE = 5200 # Cropping diameter, the larger the fewer no data

# SET SCALING if
SCALING = 1

#### Fixed Settings
* These Settings can be kept

In [ ]:
CODE_DIR = pwd
MIPPS_DIR = r'C:\Program Files\DLR MACS-Box\bin'
MIPPS_BIN = r'..\tools\MACS\mipps.exe'
EXIF_PATH = Path(CODE_DIR / Path(r'exiftool\exiftool.exe'))
mipps_script_dir = Path('mipps_scripts')

In [ ]:
mipps_script_nir = '33552_all_taps_2018-09-26_12-58-15_modelbased.mipps'
mipps_script_right = '33576_all_taps_2018-09-26_13-13-43_modelbased.mipps'
mipps_script_left = '33577_all_taps_2018-09-26_13-21-24_modelbased.mipps'

#mipps_script_nir = r'mipps_scripts\33552_all_taps_2018-09-26_12-58-15_modelbased_x4.mipps'
#mipps_script_right = r'mipps_scripts\33576_all_taps_2018-09-26_13-13-43_modelbased_x4.mipps'
#mipps_script_left = r'mipps_scripts\33577_all_taps_2018-09-26_13-21-24_modelbased_x4.mipps'

mipps_script_nir = pwd / mipps_script_dir / mipps_script_nir
mipps_script_right = pwd / mipps_script_dir / mipps_script_right
mipps_script_left = pwd / mipps_script_dir / mipps_script_left

In [ ]:
DATA_DIR = Path(PROJECT_DIR) / '01_rawdata' / 'tif'
OUTDIR = {'right': DATA_DIR / Path('33576_Right'),
          'left':DATA_DIR / Path('33577_Left'),
          'nir':DATA_DIR / Path('33552_NIR')}
tag = {'right':'MACS_RGB_Right_33576',
       'left':'MACS_RGB_Left_33577',
       'nir':'MACS_NIR_33552'}

In [ ]:
# Path of filtered footprints file (.shp file)
path_footprints = os.path.join(PROJECT_DIR, '02_studysites','footprints.shp')
outdir = os.path.join(PROJECT_DIR, '01_rawdata','tif')

#### Prepare processing dir 
* check if exists

In [ ]:
zippath = os.path.join(CODE_DIR, 'processing_folder_structure_template.zip')
nav_script_path = os.path.join(CODE_DIR, 'pix4dnav.py')

In [ ]:
with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(PROJECT_DIR)
shutil.copy(nav_script_path, outdir)

### Manual Selection of footprints 

Now select footprints and export selection as ***footprints.shp*** to ***02_footprints*** in your working directory

#### Load filtered footprints file 

In [134]:
#df_final = prepare_df_for_mipps(path_footprints, path_infiles)
df_final = prepare_df_for_mipps(path_footprints, path_infiles)

#### Workaround to deal with spaces in path" 

In [135]:
df_final['full_path'] = df_final.apply(lambda x: f'"{x.full_path}"', axis=1)

In [136]:
print("Total number of images:", len(df_final))
print("NIR images:", (df_final['Looking'] == 'center').sum())
print("RGB right images:", (df_final['Looking'] == 'right').sum())
print("RGB left images:", (df_final['Looking'] == 'left').sum())

Total number of images: 499
NIR images: 206
RGB right images: 182
RGB left images: 111


#### Run Process 

In [137]:
os.chdir(MIPPS_DIR)

In [138]:
max_roll = 3 # Select maximum roll angle to avoid image issues - SET in main settings part?
chunksize = 20 # this is a mipps-script thing

In [ ]:
# this is relevant for NIR only
if 'nir' in sensors:
    looking = 'center'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_nir = df_final[q]
    print(len(df_nir))
    for df in tqdm.tqdm_notebook(np.array_split(df_nir, len(df_nir) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_nir} -o={outdir} -j=4 {outlist}'
        os.system(s)
        #print(s)

In [ ]:
# this is RGB
if 'right' in sensors:
    looking = 'right'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_right = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_right, len(df_right) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_right} -o={outdir} -j=4 {outlist}'
        os.system(s)

In [ ]:
if 'left' in sensors:
    looking = 'left'
    q = (np.abs(df_final['Roll[deg]']) < max_roll) & (df_final['Looking'] == looking)
    df_left = df_final[q]
    for df in tqdm.tqdm_notebook(np.array_split(df_left, len(df_left) // chunksize)):
        outlist = ' '.join(df['full_path'].values[:])
        s = f'{MIPPS_BIN} -c={mipps_script_left} -o={outdir} -j=4 {outlist}'
        os.system(s)

### Rescale image values 

#### Image Statistics 

In [141]:
if SCALING:
    %time df_stats = get_image_stats_multi(OUTDIR, sensors, nth_images=10, quiet=True)
    #absolute
    scale_upper = int(df_stats['max'].mean().round())
    scale_lower = int(df_stats['min'].mean().round())
    print(f'Mean of minimums: {scale_lower}')
    print(f'Mean of maximums: {scale_upper}')

CPU times: total: 125 ms
Wall time: 3.34 s
Mean of minimums: 2548
Mean of maximums: 29623


#### Run scaling
* minimum default to 0
* consitency for final index calculation

In [142]:
if SCALING:
    n_jobs = 50
    for sensor in sensors:
        print(f'Processing {sensor}')
        #shutter_factor
        images = list(OUTDIR[sensor].glob('*.tif'))[:]
        if sensor in ['right', 'left']:
            shutter_factor = get_shutter_factor(OUTDIR, sensors)
            print(f'RGB to NIR factor = {shutter_factor}')
        else:
            shutter_factor = 1
        %time _ = Parallel(n_jobs=n_jobs)(delayed(write_new_values)(image, 0, scale_upper, shutter_factor=shutter_factor, tag=True) for image in tqdm.tqdm_notebook(images[:30]))

Processing left
RGB to NIR factor = 1.0


<timed exec>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: total: 344 ms
Wall time: 14.2 s
Processing right
RGB to NIR factor = 1.0


  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: total: 93.8 ms
Wall time: 11.4 s
Processing nir


  0%|          | 0/30 [00:00<?, ?it/s]

CPU times: total: 78.1 ms
Wall time: 3.17 s


#### Crop Corners of images 

* joblib will crash
* in other notebook it did not crash

In [143]:
if CROP_CORNER:
    #mask = make_mask((3232, 4864), disksize=DISK_SIZE)
    for sensor in sensors[:]:
        mask = make_mask((3232, 4864), disksize=DISK_SIZE)
        images = list(OUTDIR[sensor].glob('*'))
        if sensor != 'nir':
            mask = np.r_[[mask]*3]
        %time _ = [mask_and_tag(image, mask, tag=None) for image in tqdm.tqdm_notebook(images)]
        #%time _ = Parallel(n_jobs=4)(delayed(mask_and_tag)(image, mask, tag=None) for image in tqdm.tqdm_notebook(images))

#### Write exif information into all images 

In [144]:
for sensor in tqdm.tqdm_notebook(sensors):
    print(sensor)
    %time write_exif(OUTDIR[sensor], tag[sensor], EXIF_PATH)

C:\Users\initze\AppData\Local\Temp\ipykernel_31688\2113234875.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sensor in tqdm.tqdm_notebook(sensors):


  0%|          | 0/3 [00:00<?, ?it/s]

left
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Left_33577" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_non_cropped\01_rawdata\tif\33577_Left
CPU times: total: 0 ns
Wall time: 17.9 s
right
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_RGB_Right_33576" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_non_cropped\01_rawdata\tif\33576_Right
CPU times: total: 0 ns
Wall time: 32.3 s
nir
D:\pix4d_Processing\MACS_tools\exiftool\exiftool.exe -overwrite_original -Model="MACS_NIR_33552" D:\pix4d_Processing\ThawTrendAir_2019\Image_Test_non_cropped\01_rawdata\tif\33552_NIR
CPU times: total: 0 ns
Wall time: 13.2 s


#### Nav

In [145]:
navfile = list(Path(path_infiles).glob('*nav.txt'))[0]

In [146]:
shutil.copy(navfile, OUTDIR['nir'].parent / 'nav.txt')

WindowsPath('D:/pix4d_Processing/ThawTrendAir_2019/Image_Test_non_cropped/01_rawdata/tif/nav.txt')

In [147]:
os.chdir(OUTDIR['nir'].parent)
os.system('python pix4dnav.py')

0

### Run test to calculate and compare stats of every n-th image
* reduce calculation
* simplify

#### Show statistics of subsets 

In [93]:
print('Absolute Minimum')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print (df_tmp['min'].min())
print('Absolute Mean')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['mean'].mean())
print('Absolute Max')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['max'].max())
print('Mean Max')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['max'].mean())
print('Minimum perc01')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['p01'].min())
print('maximum perc99')
for df_tmp in [df_20, df_10, df_05, df_01]:
    print(df_tmp['p99'].max())

Absolute Minimum
1096.0
1067.0
909.0
280.0
Absolute Mean
10328.739980442293
10158.951161013138
10083.079676390404
10068.576469464411
Absolute Max
42582.0
43915.0
44903.0
46381.0
Mean Max
29318.423076923078
29622.52
29664.76767676768
29551.909650924026
Minimum perc01
2375.0
2358.0
2330.0
2330.0
maximum perc99
30529.0
31220.0
33022.0
33075.0


In [87]:
%time df_01 = get_image_stats_multi(OUTDIR, sensors, nth_images=1, quiet=True)

CPU times: total: 1.48 s
Wall time: 16.6 s


In [88]:
%time df_05 = get_image_stats_multi(OUTDIR, sensors, nth_images=5, quiet=True)

CPU times: total: 469 ms
Wall time: 7.55 s


In [90]:
%time df_20 = get_image_stats_multi(OUTDIR, sensors, nth_images=20, quiet=True)

CPU times: total: 93.8 ms
Wall time: 2.81 s


In [102]:
#RGB
scale_upper = df_20.query('sensor in ("left", "right")')['max'].mean().round()
scale_lower = df_20.query('sensor in ("left", "right")')['min'].mean().round()
print(scale_lower, scale_upper)

In [104]:
#NIR
scale_upper = df_20.query('sensor in ("nir")')['max'].mean().round()
scale_lower = df_20.query('sensor in ("nir")')['min'].mean().round()
print(scale_lower, scale_upper)

3966.0 34761.0
